<a href="https://colab.research.google.com/github/Yazan-Daibes/Arabic-Toxic-Comment-Classification/blob/main/AraBERT_and_Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
import nltk
nltk.download("all")
import matplotlib.pyplot as plt
import torch

%matplotlib inline

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import string
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report
from numpy import concatenate
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.semi_supervised import LabelPropagation

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import numpy as np
import tensorflow as tf

In [ ]:
#!pip uninstall xlrd -y
#!pip install --upgrade pandas
#!pip install 'xlrd == 1.2.0'
!pip install openpyxl 
import os
import io
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
data = pd.read_csv('/content/shuffled_4_labels.csv')
print(data)

                                                 comment    label
0                       اللهم عليك بالحكام العرب و راموس   normal
1                         اسال امك وهى بتتجلد وهى بتتناك  obscene
2      وقد عبر عن ذلك بشكل علني ومباشر وزير الخارجية ...   normal
3      خلص كول هوا العمى بقلبك شو حاقد و وسخ الله يحش...  abusive
4                   لا تغلط هذا صاروخ ادبي بعيد المدى      normal
...                                                  ...      ...
15774                كيف هالحديث قلك في واحد عاهر القضاء  abusive
15775                                         اتفوه عليك  obscene
15776  لي اصلا اكتر بلد عربي قراراته داخلية مية بالمي...   normal
15777  يا كلب يا خنزير تراقب مكوة امك  القحبه  وانت ي...     hate
15778  عزيزي وجهة نظرك هي وجهة نظري هذه الارض من البح...   normal

[15779 rows x 2 columns]


In [ ]:
data = data.rename( columns = {'comment':'Tweet','label':'Class'} , inplace= False )

In [ ]:
data['Class'].value_counts()

normal     8215
abusive    3475
obscene    2485
hate       1604
Name: Class, dtype: int64

In [ ]:
#data["Class"].replace({"normal":"0","abusive":"1","hate":"2","hateful":"2","abusive_hateful":"2","hateful_disrespectful":"2"},inplace=True)
#data["Class"].replace({"normal":"0","abusive":"1","hate":"2","hateful":"2","abusive_hateful":"2","hateful_disrespectful":"2","obscene":"3"},inplace=True)
data["Class"].replace({"normal":"0","abusive":"1","hate":"1","obscene":"1"},inplace=True)



In [ ]:
# test_data = data.sample(frac=0.1,random_state=42)
# test_data.shape
# data=data.drop(test_data.index)
# data.shape,test_data.shape

In [ ]:
data.rename(columns={'Tweet':'texts'}, inplace=True)
# test_data.rename(columns={'Tweet':'texts'}, inplace=True)
data.rename(columns={'Class':'data_labels'}, inplace=True)
# test_data.rename(columns={'Class':'data_labels'}, inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split

texts = data.texts.values
labels = data.data_labels.values
# X_val= test_data.texts.values
# Y_val = test_data.data_labels.values

In [ ]:
# def load_text(path):
#     """Load text data, lowercase text and save to a list."""

#     with open(path, 'rb') as f:
#         texts = []
#         for line in f:
#             texts.append(line.decode(errors='ignore').lower().strip())

#     return texts

# # Load files
# neg_text = load_text('Data/rt-polaritydata/rt-polarity.neg')
# pos_text = load_text('Data/rt-polaritydata/rt-polarity.pos')

# # Concatenate and label data
# texts = np.array(neg_text + pos_text)
# labels = np.array([0]*len(neg_text) + [1]*len(pos_text))

In [ ]:
# %%time
# URL = "https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip"
# FILE = "/content/cc.sq.300.vec"

# if os.path.isdir(FILE):
#     print("fastText exists.")
# else:
#     !wget -P $FILE $URL
#     !unzip $FILE/crawl-300d-2M.vec.zip -d $FILE

In [ ]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla T4


In [ ]:
from nltk.tokenize import word_tokenize
from collections import defaultdict

def tokenize(texts):
    """Tokenize texts, build vocabulary and find maximum sentence length.
    
    Args:
        texts (List[str]): List of text data
    
    Returns:
        tokenized_texts (List[List[str]]): List of list of tokens
        word2idx (Dict): Vocabulary built from the corpus
        max_len (int): Maximum sentence length
    """

    max_len = 0
    tokenized_texts = []
    word2idx = {}

    # Add <pad> and <unk> tokens to the vocabulary
    word2idx['<pad>'] = 0
    word2idx['<unk>'] = 1

    # Building our vocab from the corpus starting from index 2
    idx = 2
    for sent in texts:
        tokenized_sent = word_tokenize(sent)

        # Add `tokenized_sent` to `tokenized_texts`
        tokenized_texts.append(tokenized_sent)

        # Add new token to `word2idx`
        for token in tokenized_sent:
            if token not in word2idx:
                word2idx[token] = idx
                idx += 1

        # Update `max_len`
        max_len = max(max_len, len(tokenized_sent))

    return tokenized_texts, word2idx, max_len

def encode(tokenized_texts, word2idx, max_len):
    """Pad each sentence to the maximum sentence length and encode tokens to
    their index in the vocabulary.

    Returns:
        input_ids (np.array): Array of token indexes in the vocabulary with
            shape (N, max_len). It will the input of our CNN model.
    """

    input_ids = []
    for tokenized_sent in tokenized_texts:
        # Pad sentences to max_len
        tokenized_sent += ['<pad>'] * (max_len - len(tokenized_sent))

        # Encode tokens to input_ids
        input_id = [word2idx.get(token) for token in tokenized_sent]
        input_ids.append(input_id)
    
    return np.array(input_ids)

In [ ]:
from tqdm import tqdm_notebook

def load_pretrained_vectors(word2idx, fname):
    """Load pretrained vectors and create embedding layers.
    
    Args:
        word2idx (Dict): Vocabulary built from the corpus
        fname (str): Path to pretrained vector file

    Returns:
        embeddings (np.array): Embedding matrix with shape (N, d) where N is
            the size of word2idx and d is embedding dimension
    """

    print("Loading pretrained vectors...")
    fin = open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())

    # Initilize random embeddings
    embeddings = np.random.uniform(-0.25, 0.25, (len(word2idx), d))
    embeddings[word2idx['<pad>']] = np.zeros((d,))

    # Load pretrained vectors
    count = 0
    for line in tqdm_notebook(fin):
        tokens = line.rstrip().split(' ')
        word = tokens[0]
        if word in word2idx:
            count += 1
            embeddings[word2idx[word]] = np.array(tokens[1:], dtype=np.float32)

    print(f"There are {count} / {len(word2idx)} pretrained vectors found.")

    return embeddings

In [ ]:
# Tokenize, build vocabulary, encode tokens
print("Tokenizing...\n")
tokenized_texts, word2idx, max_len = tokenize(texts)
print(word2idx)
input_ids = encode(tokenized_texts, word2idx, max_len)

# Load pretrained vectors
embeddings = load_pretrained_vectors(word2idx, "/content/drive/MyDrive/cc.ar.300.vec")
embeddings = torch.tensor(embeddings)

Tokenizing...

{'<pad>': 0, '<unk>': 1, 'اللهم': 2, 'عليك': 3, 'بالحكام': 4, 'العرب': 5, 'و': 6, 'راموس': 7, 'اسال': 8, 'امك': 9, 'وهى': 10, 'بتتجلد': 11, 'بتتناك': 12, 'وقد': 13, 'عبر': 14, 'عن': 15, 'ذلك': 16, 'بشكل': 17, 'علني': 18, 'ومباشر': 19, 'وزير': 20, 'الخارجية': 21, 'جبران': 22, 'باسيل': 23, 'في': 24, 'كلمته': 25, 'التي': 26, 'حصدت': 27, 'اجماعا': 28, 'عربيا': 29, 'القمة': 30, 'العربية': 31, 'التنموية': 32, 'خلص': 33, 'كول': 34, 'هوا': 35, 'العمى': 36, 'بقلبك': 37, 'شو': 38, 'حاقد': 39, 'وسخ': 40, 'الله': 41, 'يحشرك': 42, 'مع': 43, 'يزيد': 44, 'اللي': 45, 'علمك': 46, 'هالحقد': 47, 'لا': 48, 'تغلط': 49, 'هذا': 50, 'صاروخ': 51, 'ادبي': 52, 'بعيد': 53, 'المدى': 54, 'يلعن': 55, 'الي': 56, 'نفضك': 57, 'خاين': 58, 'كلب': 59, 'خرا': 60, 'أقل': 61, 'بل': 62, 'بول': 63, 'وخرا': 64, 'وكان': 65, 'يقوم': 66, 'بجمع': 67, 'الدولارات': 68, 'الجدبدة': 69, 'ويستبدل': 70, 'البالية': 71, 'صاحبة': 72, 'الرائحة': 73, 'النتنة': 74, 'أخبار': 75, 'المرحاض': 76, 'كيف': 77, 'يعني': 78, 'حرم': 79, 'ال

<ipython-input-74-b47afdb8d107>:25: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(fin):


0it [00:00, ?it/s]

There are 25234 / 30943 pretrained vectors found.


In [ ]:
from torch.utils.data import (TensorDataset, DataLoader, RandomSampler,
                              SequentialSampler)

def data_loader(train_inputs, val_inputs, train_labels, val_labels,
                batch_size=16):
    """Convert train and validation sets to torch.Tensors and load them to
    DataLoader.
    """

    # Convert data type to torch.Tensor
    train_inputs, val_inputs, train_labels, val_labels =\
    tuple(torch.tensor(data) for data in
          [train_inputs, val_inputs, train_labels, val_labels])

    # Specify batch_size
    batch_size = 16

    # Create DataLoader for training data
    train_data = TensorDataset(train_inputs, train_labels)
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

    # Create DataLoader for validation data
    val_data = TensorDataset(val_inputs, val_labels)
    val_sampler = SequentialSampler(val_data)
    val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

    return train_dataloader, val_dataloader

In [ ]:
from sklearn.model_selection import train_test_split

# Train Test Split
train_inputs, val_inputs, train_labels, val_labels = train_test_split(
    input_ids, labels, test_size=0.1, random_state=42)

# Load data to PyTorch DataLoader
train_dataloader, val_dataloader = data_loader(train_inputs, val_inputs, train_labels.astype(int), val_labels.astype(int), batch_size=16)


In [ ]:
# Sample configuration:
filter_sizes = [2, 3, 4]
num_filters = [2, 2, 2]

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN_NLP(nn.Module):
    """An 1D Convulational Neural Network for Sentence Classification."""
    def __init__(self,
                 pretrained_embedding=None,
                 freeze_embedding=False,
                 vocab_size=None,
                 embed_dim=300,
                 filter_sizes=[3, 4, 5],
                 num_filters=[100, 100, 100],
                 num_classes=2,
                 dropout=0.5):
        """
        The constructor for CNN_NLP class.

        Args:
            pretrained_embedding (torch.Tensor): Pretrained embeddings with
                shape (vocab_size, embed_dim)
            freeze_embedding (bool): Set to False to fine-tune pretraiend
                vectors. Default: False
            vocab_size (int): Need to be specified when not pretrained word
                embeddings are not used.
            embed_dim (int): Dimension of word vectors. Need to be specified
                when pretrained word embeddings are not used. Default: 300
            filter_sizes (List[int]): List of filter sizes. Default: [3, 4, 5]
            num_filters (List[int]): List of number of filters, has the same
                length as `filter_sizes`. Default: [100, 100, 100]
            n_classes (int): Number of classes. Default: 2
            dropout (float): Dropout rate. Default: 0.5
        """

        super(CNN_NLP, self).__init__()
        # Embedding layer
        if pretrained_embedding is not None:
            self.vocab_size, self.embed_dim = pretrained_embedding.shape
            self.embedding = nn.Embedding.from_pretrained(pretrained_embedding,
                                                          freeze=freeze_embedding)
        else:
            self.embed_dim = embed_dim
            self.embedding = nn.Embedding(num_embeddings=vocab_size,
                                          embedding_dim=self.embed_dim,
                                          padding_idx=0,
                                          max_norm=5.0)
        # Conv Network
        self.conv1d_list = nn.ModuleList([
            nn.Conv1d(in_channels=self.embed_dim,
                      out_channels=num_filters[i],
                      kernel_size=filter_sizes[i])
            for i in range(len(filter_sizes))
        ])
        # Fully-connected layer and Dropout
        self.fc = nn.Linear(np.sum(num_filters), num_classes)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, input_ids):
        """Perform a forward pass through the network.

        Args:
            input_ids (torch.Tensor): A tensor of token ids with shape
                (batch_size, max_sent_length)

        Returns:
            logits (torch.Tensor): Output logits with shape (batch_size,
                n_classes)
        """

        # Get embeddings from `input_ids`. Output shape: (b, max_len, embed_dim)
        x_embed = self.embedding(input_ids).float()

        # Permute `x_embed` to match input shape requirement of `nn.Conv1d`.
        # Output shape: (b, embed_dim, max_len)
        x_reshaped = x_embed.permute(0, 2, 1)

        # Apply CNN and ReLU. Output shape: (b, num_filters[i], L_out)
        x_conv_list = [F.relu(conv1d(x_reshaped)) for conv1d in self.conv1d_list]

        # Max pooling. Output shape: (b, num_filters[i], 1)
        x_pool_list = [F.max_pool1d(x_conv, kernel_size=x_conv.shape[2])
            for x_conv in x_conv_list]
        
        # Concatenate x_pool_list to feed the fully connected layer.
        # Output shape: (b, sum(num_filters))
        x_fc = torch.cat([x_pool.squeeze(dim=2) for x_pool in x_pool_list],
                         dim=1)
        
        # Compute logits. Output shape: (b, n_classes)
        logits = self.fc(self.dropout(x_fc))

        return logits

In [ ]:
import torch.optim as optim

def initilize_model(pretrained_embedding=None,
                    freeze_embedding=False,
                    vocab_size=None,
                    embed_dim=300,
                    filter_sizes=[3, 4, 5],
                    num_filters=[100, 100, 100],
                    num_classes=2,
                    dropout=0.5,
                    learning_rate=0.01):
    """Instantiate a CNN model and an optimizer."""

    assert (len(filter_sizes) == len(num_filters)), "filter_sizes and \
    num_filters need to be of the same length."

    # Instantiate CNN model
    cnn_model = CNN_NLP(pretrained_embedding=pretrained_embedding,
                        freeze_embedding=freeze_embedding,
                        vocab_size=vocab_size,
                        embed_dim=embed_dim,
                        filter_sizes=filter_sizes,
                        num_filters=num_filters,
                        num_classes=2,
                        dropout=0.5)
    
    # Send model to `device` (GPU/CPU)
    cnn_model.to(device)

    # Instantiate Adadelta optimizer
    optimizer = optim.Adadelta(cnn_model.parameters(),
                               lr=learning_rate,
                               rho=0.95)

    return cnn_model, optimizer

In [ ]:
import random
import time

# Specify loss function
loss_fn = nn.CrossEntropyLoss()

def set_seed(seed_value=42):
    """Set seed for reproducibility."""

    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def train(model, optimizer, train_dataloader, val_dataloader=None, epochs=10):
    """Train the CNN model."""
    
    # Tracking best validation accuracy
    best_accuracy = 0

    # Start training loop
    print("Start training...\n")
    print(f"{'Epoch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Elapsed':^9}")
    print("-"*60)

    for epoch_i in range(epochs):
        # =======================================
        #               Training
        # =======================================

        # Tracking time and loss
        t0_epoch = time.time()
        total_loss = 0

        # Put the model into the training mode
        model.train()

        for step, batch in enumerate(train_dataloader):
            # Load batch to GPU
            b_input_ids, b_labels = tuple(t.to(device) for t in batch)

            # Zero out any previously calculated gradients
            model.zero_grad()

            # Perform a forward pass. This will return logits.
            logits = model(b_input_ids)

            # Compute loss and accumulate the loss values
            loss = loss_fn(logits, b_labels)
            total_loss += loss.item()

            # Perform a backward pass to calculate gradients
            loss.backward()

            # Update parameters
            optimizer.step()

        # Calculate the average loss over the entire training data
        avg_train_loss = total_loss / len(train_dataloader)

        # =======================================
        #               Evaluation
        # =======================================
        if val_dataloader is not None:
            # After the completion of each training epoch, measure the model's
            # performance on our validation set.
            val_loss, val_accuracy = evaluate(model, val_dataloader)

            # Track the best accuracy
            if val_accuracy > best_accuracy:
                best_accuracy = val_accuracy

            # Print performance over the entire training data
            time_elapsed = time.time() - t0_epoch
            print(f"{epoch_i + 1:^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")

        torch.save(model, './model_word2vec.h5')    
    print("\n")
    torch.save(model, './model_word2vec.h5')
    print(f"Training complete! Best accuracy: {best_accuracy:.2f}%.")

def evaluate(model, val_dataloader):
    """After the completion of each training epoch, measure the model's
    performance on our validation set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled
    # during the test time.
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_loss = []
    # For each batch in our validation set...
    for batch in val_dataloader:
        # Load batch to GPU
        b_input_ids, b_labels = tuple(t.to(device) for t in batch)

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids)

        # Compute loss
        loss = loss_fn(logits, b_labels)
        val_loss.append(loss.item())

        # Get the predictions
        preds = torch.argmax(logits, dim=1).flatten()
        # Calculate the accuracy rate
        accuracy = (preds == b_labels).cpu().numpy().mean() * 100
        val_accuracy.append(accuracy)

    # Compute the average accuracy and loss over the validation set.
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)

    return val_loss, val_accuracy

In [ ]:
# CNN-rand: Word vectors are randomly initialized.
# set_seed(42)
# cnn_rand, optimizer = initilize_model(vocab_size=len(word2idx),
#                                       embed_dim=300,
#                                       learning_rate=0.25,
#                                       dropout=0.5)
# train(cnn_rand, optimizer, train_dataloader, val_dataloader, epochs=20)

In [ ]:
# # CNN-static: fastText pretrained word vectors are used and freezed during training.
# set_seed(42)
# cnn_static, optimizer = initilize_model(pretrained_embedding=embeddings,
#                                         freeze_embedding=True,
#                                         learning_rate=0.25,
#                                         dropout=0.5)
# train(cnn_static, optimizer, train_dataloader, val_dataloader, epochs=20)

In [ ]:
# CNN-non-static: fastText pretrained word vectors are fine-tuned during training.
set_seed(42)
cnn_non_static, optimizer = initilize_model(pretrained_embedding=embeddings,
                                            freeze_embedding=False,
                                            learning_rate=0.25,
                                            dropout=0.5)
train(cnn_non_static, optimizer, train_dataloader, val_dataloader, epochs=100)

RuntimeError: ignored

In [ ]:
from IPython.display import FileLink
FileLink(r'model_word2vec.h5') 

In [ ]:
import torch
model2= torch.load('model_word2vec.h5')

In [ ]:
val_loss, val_accuracy = evaluate(model2, val_dataloader)
print(val_accuracy)

In [ ]:
import torch.nn.functional as F

def bert_predict(model2, test_dataloader):
    """Perform a forward pass on the trained BERT model to predict probabilities
    on the test set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model2.eval()

    all_logits = []

    # For each batch in our test set...
    for batch in test_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask = tuple(t.to(device) for t in batch)
        # Compute logits
        with torch.no_grad():
            logits = model2(b_input_ids)

        all_logits.append(logits)
    
    # Concatenate logits from each batch
    all_logits = torch.cat(all_logits, dim=0)

    # Apply softmax to calculate probabilities
    probs = F.softmax(all_logits, dim=1).cpu().numpy()

    return probs

In [ ]:
probs = bert_predict(model2, val_dataloader)
print(probs)


In [ ]:
preds=probs[:,1]
print(len(probs))
print(len(val_labels))

In [ ]:
multi_class_preds = probs.argmax(axis=1)
print(multi_class_preds)
c = 0
for i in range(len(probs)):
  if int(val_labels[i]) == multi_class_preds[i]:
    c += 1
print(c)

In [ ]:
import os
import pandas as pd
import sklearn
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn import metrics


In [ ]:
sklearn.metrics.confusion_matrix(val_labels.astype(int), multi_class_preds,  labels=None, sample_weight=None, normalize=None)

In [ ]:
array = pd.crosstab(val_labels.astype(int), multi_class_preds, rownames=['Actual'], colnames=['Predicted'], margins=True)
array

# **convert the table into a confusion matrix display**

In [ ]:
print(classification_report(val_labels.astype(int), multi_class_preds))

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df_cm = pd.DataFrame(array, range(2), range(2))
# plt.figure(figsize=(10,7))
sn.set(font_scale=1.4) # for label size
sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}) # font size

In [ ]:
#!pip install --upgrade transformers

In [ ]:
class_names =["normal","abusive","hate","obscene"]

In [ ]:
def predict(text, model2=model2, max_len=62):
    """Predict probability that a review is positive."""

    # Tokenize, pad and encode text
    #tokenized_texts, word2idx, max_len = tokenize(text)
    max_len = 512
    tokens = word_tokenize(text.lower())
    padded_tokens = tokens + ['<pad>'] * (max_len - len(tokens))
    input_id = [word2idx.get(token, word2idx['<unk>']) for token in padded_tokens]

    # Convert to PyTorch tensors
    input_id = torch.tensor(input_id).unsqueeze(dim=0).to(device)

    # Compute logits
    logits = model2(input_id).to(device)

    #  Compute probability
    probs = F.softmax(logits, dim=1).squeeze(dim=0)
    _, prediction = torch.max(logits, dim=1)
    #print(f"This review1 is {probs[0] * 100:.2f}% normal.")
    #print(f"This review2 is {probs[1] * 100:.2f}% abusive.")
    #print(f"This review3 is {probs[2] * 100:.2f}% hate.")
    #print(f"This review4 is {probs[3] * 100:.2f}% obscene.")
    
    return class_names[prediction]


In [ ]:
#predict("لكردي كلب من يوم يومه بحاجة دائما لسيد وهلا اصبحتم خنازير اسرائيل بمرتبة الكلب")
#predict("يازلمي أنت شو بتتعاطى. على الأرجح بول البعير")
#predict("ما تفكر كل الناس متلكم يا شاربي بول البعير")
#predict("صوتك كتير جميل")
#predict("سانك حصانك وحسنا فعلت قطر لتلغي مركز الأبحاث لا مرحبا بكم انتم ولا تستاهلون اي عمل لكم ناكرين المعروف")
#predict("الله ياخدكم")
#predict("يجي يوم وسننتقم من كل كلب لبناني تسبب بأذى للشعب السوري وسيدفعون الثمن غال")

# c = 0
# k = 0
# for i in X_val:
#   if class_names[int(Y_val[k])] == predict(i):
#     c += 1
#   k += 1
# print((c/len(X_val))*100)
# print(len(X_val))
# print(len(texts))